# Importing required libraries

In [1]:
!pip install pytorch_lightning
!pip install transformers==4.0.0rc1

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


     |▎                               | 10 kB 830 kB/s eta 0:00:02

     |▊                               | 30 kB 522 kB/s eta 0:00:03

     |█▌                              | 61 kB 268 kB/s eta 0:00:05

     |██▏                             | 92 kB 333 kB/s eta 0:00:04

...

    Found existing installation: transformers 3.4.0


    Uninstalling transformers-3.4.0:


      Successfully uninstalled transformers-3.4.0


ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

allennlp 1.2.1 requires transformers<3.5,>=3.1, but you'll have transformers 4.0.0rc1 which is incompatible.


You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
import argparse
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
import os
from transformers import (
    AdamW,
    MT5ForConditionalGeneration,
    AutoTokenizer,
    get_linear_schedule_with_warmup
)

# Setting up wandb

In [3]:
import wandb
#wandb.login('3aa5adb8223166fea653d0941937427288afdf37')
!wandb login 3aa5adb8223166fea653d0941937427288afdf37

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


# Setting seeds

In [4]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

In [5]:
class T5FineTuner(pl.LightningModule):
    def __init__(self, hparams):
        super(T5FineTuner, self).__init__()
        self.hparams   = hparams
        self.model     = MT5ForConditionalGeneration.from_pretrained(hparams.model_name_or_path)
        self.tokenizer = AutoTokenizer.from_pretrained(hparams.tokenizer_name_or_path)
        wandb.init(project="transformers_ParsParaphraser")
        wandb.watch(self.model, log="all")

    def is_logger(self):
        return True

    def forward(self, input_ids, attention_mask=None, decoder_input_ids=None, decoder_attention_mask=None, labels=None):
        return self.model(
            input_ids,
            attention_mask=attention_mask,
            decoder_input_ids=decoder_input_ids,
            decoder_attention_mask=decoder_attention_mask,
            labels=labels,
        )

    def _step(self, batch):
        labels = batch["target_ids"]
        labels[labels[:, :] == self.tokenizer.pad_token_id] = -100

        outputs = self(
            input_ids=batch["source_ids"],
            attention_mask=batch["source_mask"],
            labels=labels,
            decoder_attention_mask=batch['target_mask']
        )

        loss = outputs[0]

        return loss

    def training_step(self, batch, batch_idx):
        loss = self._step(batch)

        tensorboard_logs = {"train_loss": loss}
        wandb.log({"train_loss": loss})
        return {"loss": loss, "log": tensorboard_logs}

    def training_epoch_end(self, outputs):
        avg_train_loss = torch.stack([x["loss"] for x in outputs]).mean()
        tensorboard_logs = {"avg_train_loss": avg_train_loss}
        wandb.log({"avg_train_loss": avg_train_loss})
        values =  {"avg_train_loss": avg_train_loss, "log": tensorboard_logs, 'progress_bar': tensorboard_logs}
        self.log_dict(values)

    def validation_step(self, batch, batch_idx):
        loss = self._step(batch)
        self.log("val_loss",loss)

    def validation_epoch_end(self, outputs):
        #avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
        avg_loss  = 0
        tensorboard_logs = {"val_loss": avg_loss}
        wandb.log({"val_loss": avg_loss})
        values =  {"avg_val_loss": avg_loss, "log": tensorboard_logs, 'progress_bar': tensorboard_logs}
        self.log_dict(values)

    def configure_optimizers(self):
        "Prepare optimizer and schedule (linear warmup and decay)"

        model = self.model
        no_decay = ["bias", "LayerNorm.weight"]
        optimizer_grouped_parameters = [
            {
                "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
                "weight_decay": self.hparams.weight_decay,
            },
            {
                "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
                "weight_decay": 0.0,
            },
        ]
        optimizer = AdamW(optimizer_grouped_parameters, lr=self.hparams.learning_rate, eps=self.hparams.adam_epsilon)
        self.opt = optimizer
        return [optimizer]

    def optimizer_step(self,epoch, batch_idx, optimizer, optimizer_idx,optimizer_closure, on_tpu, using_native_amp, using_lbfgs):
        if self.trainer.use_tpu:
            xm.optimizer_step(optimizer)
        else:
            optimizer.step(closure=optimizer_closure)
        optimizer.zero_grad()
        self.lr_scheduler.step()

    def get_tqdm_dict(self):
        tqdm_dict = {"loss": "{:.3f}".format(self.trainer.avg_loss), "lr": self.lr_scheduler.get_last_lr()[-1]}

        return tqdm_dict

    def train_dataloader(self):
        train_dataset = self.get_dataset(dataset_type = 'train', train_size=0.8)
        dataloader    = DataLoader(train_dataset, batch_size=self.hparams.train_batch_size, drop_last=True, shuffle=True,num_workers=2)
        t_total       = (
                (len(dataloader.dataset) // (self.hparams.train_batch_size * max(1, self.hparams.n_gpu)))
                // self.hparams.gradient_accumulation_steps
                * float(self.hparams.num_train_epochs)
        )
        scheduler = get_linear_schedule_with_warmup(
            self.opt, num_warmup_steps=self.hparams.warmup_steps, num_training_steps=t_total
        )
        self.lr_scheduler = scheduler
        return dataloader

    def val_dataloader(self):
        val_dataset = self.get_dataset(dataset_type = 'test', train_size=0.8)
        return DataLoader(val_dataset, batch_size=self.hparams.eval_batch_size, num_workers=2)
    
    def get_dataset(self, dataset_type, train_size=0.8):
        df            = pd.read_csv(self.hparams.data_dir)
        train_dataset = df.sample(frac=train_size,random_state = self.hparams.seed)
        val_dataset   = df.drop(train_dataset.index).reset_index(drop=True)
        train_dataset = train_dataset.reset_index(drop=True)
        if dataset_type == 'train':
            return Dataset(train_dataset, self.tokenizer ,self.hparams.source_length, self.hparams.paraphrase_length)
        if dataset_type == 'test':
            return Dataset(val_dataset,   self.tokenizer, self.hparams.source_length, self.hparams.paraphrase_length)
        else:
            return None


# Dataset Class

In [6]:
from torch.utils.data import Dataset

class Dataset(Dataset):
    def __init__(self, dataframe, tokenizer, source_len  , par_len  , transform=None):
        self.data       = dataframe
        self.transform  = transform
        self.source_len = source_len
        self.par_len    = par_len
        self.tokenizer  = tokenizer

    def __len__(self):
        return len(self.data)

    def tokenizing(self,txt_list):
      tokenized = [self.tokenizer(txt, max_length=90, truncation=True, padding='max_length', return_tensors='pt')['input_ids'] for txt in txt_list]
      if len(txt_list) == 1 : tokenized = tokenized[0]
      return tokenized
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        sentence_id = self.data.iloc[idx, 0]
        sentence    = self.data.iloc[idx, 1]
        paraphrase  = self.data.iloc[idx, 2]

        source = self.tokenizer.batch_encode_plus([sentence],    max_length= self.source_len, truncation=True,  padding='max_length' ,return_tensors='pt')
        target = self.tokenizer.batch_encode_plus([paraphrase],  max_length= self.par_len,    truncation=True,  padding='max_length'  ,return_tensors='pt')

        source_ids = source['input_ids'].squeeze()
        source_mask = source['attention_mask'].squeeze()
        target_ids = target['input_ids'].squeeze()
        target_mask = target['attention_mask'].squeeze()

        if self.transform:
            sentence = self.transform(sentence)

        return {
            'source_ids':   source_ids.to(dtype=torch.long), 
            'source_mask':  source_mask.to(dtype=torch.long), 
            'target_ids':   target_ids.to(dtype=torch.long),
            'target_mask':  target_mask.to(dtype=torch.long),
        }

# logging class

In [7]:
logger = logging.getLogger(__name__)

class LoggingCallback(pl.Callback):
        def on_validation_end(self, trainer, pl_module):
            logger.info("***** Validation results *****")
            if pl_module.is_logger():
                  metrics = trainer.callback_metrics
                  # Log results
                  for key in sorted(metrics):
                    if key not in ["log", "progress_bar"]:
                      logger.info("{} = {}\n".format(key, str(metrics[key])))

        def on_test_end(self, trainer, pl_module):
            logger.info("***** Test results *****")

            if pl_module.is_logger():
                metrics = trainer.callback_metrics

                  # Log and save results to file
                output_test_results_file = os.path.join(pl_module.hparams.output_dir, "test_results.txt")
                with open(output_test_results_file, "w") as writer:
                    for key in sorted(metrics):
                          if key not in ["log", "progress_bar"]:
                            logger.info("{} = {}\n".format(key, str(metrics[key])))
                            writer.write("{} = {}\n".format(key, str(metrics[key])))

# Setting up Arguments

In [8]:
args_dict = dict(
    data_dir="../input/persian-paraphrase-dataset/Chunk_5_official.csv", # path for data files
    output_dir="./checkpoints", # path to save the checkpoints
    model_name_or_path='google/mt5-small',
    tokenizer_name_or_path='google/mt5-small',
    max_seq_length=512,
    source_length = 512,
    paraphrase_length = 150,
    learning_rate=3e-4,
    weight_decay=0.0,
    adam_epsilon=1e-8,
    warmup_steps=0,
    train_batch_size=5,
    eval_batch_size=5,
    num_train_epochs=2,
    gradient_accumulation_steps=8,
    n_gpu=1,
    #early_stop_callback=False,
    fp_16=False, # if you want to enable 16-bit training then install apex and set this to true
    opt_level='O1', # you can find out more on optimisation levels here https://nvidia.github.io/apex/amp.html#opt-levels-and-properties
    max_grad_norm=1.0, # if you enable 16-bit training then set this to a sensible value, 0.5 is a good default
    seed=42,
)

In [9]:
args = argparse.Namespace(**args_dict)
print(args_dict)

{'data_dir': '../input/persian-paraphrase-dataset/Chunk_5_official.csv', 'output_dir': './checkpoints', 'model_name_or_path': 'google/mt5-small', 'tokenizer_name_or_path': 'google/mt5-small', 'max_seq_length': 512, 'source_length': 512, 'paraphrase_length': 150, 'learning_rate': 0.0003, 'weight_decay': 0.0, 'adam_epsilon': 1e-08, 'warmup_steps': 0, 'train_batch_size': 5, 'eval_batch_size': 5, 'num_train_epochs': 2, 'gradient_accumulation_steps': 8, 'n_gpu': 1, 'fp_16': False, 'opt_level': 'O1', 'max_grad_norm': 1.0, 'seed': 42}


# Train

In [10]:
checkpoint_callback = pl.callbacks.ModelCheckpoint(
    
    period =1,filepath=args.output_dir, prefix="checkpoint", monitor="val_loss", mode="min", save_top_k=1
)

train_params = dict(
    accumulate_grad_batches=args.gradient_accumulation_steps,
    gpus=args.n_gpu,
    max_epochs=args.num_train_epochs,
    precision= 16 if args.fp_16 else 32,
    amp_level=args.opt_level,
    gradient_clip_val=args.max_grad_norm,
    checkpoint_callback=checkpoint_callback,
    callbacks=[LoggingCallback()],
)


/opt/conda/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: Checkpoint directory /kaggle/working exists and is not empty. With save_top_k=1, all files in this directory will be deleted when a checkpoint is saved!
  warnings.warn(*args, **kwargs)


In [11]:
print ("Initialize model")
model = T5FineTuner(args)

trainer = pl.Trainer(**train_params)

Initialize model


wandb: Currently logged in as: arman_n (use `wandb login --relogin` to force relogin)


wandb: wandb version 0.13.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


GPU available: True, used: True


TPU available: False, using: 0 TPU cores


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [12]:
print (" Training model")
trainer.fit(model)

print ("training finished")

print ("Saving model")
model.model.save_pretrained("/kaggle/working/result")

print ("Saved model")

 Training model



  | Name  | Type                        | Params
------------------------------------------------------
0 | model | MT5ForConditionalGeneration | 300 M 


/opt/conda/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: The {log:dict keyword} was deprecated in 0.9.1 and will be removed in 1.0.0
Please use self.log(...) inside the lightningModule instead.

# log on a step or aggregate epoch metric to the logger and/or progress bar
# (inside LightningModule)
self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
  warnings.warn(*args, **kwargs)
